# 0. 라이브러리 환경 설정, 데이터 불러오기

In [7]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# input데이터 경로
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [8]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [9]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


# 1. 데이터 분석 목적

분석 목적: “어떤 탑승객이 살아남는가?” using passenger data (ie name, age, gender, socio-economic class, etc)  
=> **train.csv** 데이터를 통해 패턴을 찾아 **test.csv** 데이터에서의 승객 생존여부를 예측
## 패턴 탐색
**gender_submission.csv**의 샘플 제출 파일은 모든 여성 승객이 살아남은 것으로 가정합니다(모든 남성 승객이 사망).
이 가정이 합리적인지 **train.csv**데이터를 통해 확인합니다. 

In [10]:
women = train_data.loc[train_data.Sex == 'female']["Survived"]
rate_women = sum(women)/len(women)

print("% of women who survived:", rate_women)

% of women who survived: 0.7420382165605095


In [11]:
men = train_data.loc[train_data.Sex == 'male']["Survived"]
rate_men = sum(men)/len(men)

print("% of men who survived:", rate_men)

% of men who survived: 0.18890814558058924


이것을 통해 탑승한 여성의 75%가 생존했다는 것을 알 수 있습니다. 반면에 남성의 19%만이 생존했다는 것을 알 수 있습니다. 성별이 뚜렷한 생존의 지표가 되는 것 같아 보이는 만큼 **gender_submission.csv**의 제출 파일은 첫 번째 추측이 나쁘지 않고 타당하다는 것을 알 수 있습니다. 

그러나 결국, 성별에 기반한 이 제출은 하나의 속성에 대해서만 예측을 합니다. 여러분이 상상할 수 있듯이, 여러 속성을 고려함으로써, 우리는 잠재적으로 더 나은 정보를 제공할 수 있는 더 복잡한 패턴을 발견할 수 있습니다. 한 번에 여러 열을 고려하는 것은 상당히 어렵기 때문에(또는 여러 다른 열에서 가능한 모든 패턴을 동시에 고려하는 것은 오랜 시간이 걸릴 수 있어서), 머신 러닝을 사용하여 이를 자동화합니다.

## machine learning model

우리는 **random forest model**이라고 알려진 모델을 만들 것입니다. 이 모델은 여러 개의 "트리"로 구성되어 있습니다(아래 그림에는 트리 3개가 있지만, 우리는 100개 트리 씁니다) 각 승객의 데이터를 개별적으로 고려하고 개인이 생존했는지 여부를 투표합니다. 그런 다음, random forest model 모델은 민주적인 결정을 내립니다: 가장 많은 표를 얻은 결과가 승리합니다!

![](https://i.imgur.com/AC9Bq63.png)

In [12]:
from sklearn.ensemble import RandomForestClassifier

y = train_data["Survived"]

features = ["Pclass", "Sex", "SibSp", "Parch"]
X = pd.get_dummies(train_data[features])
X_test = pd.get_dummies(test_data[features])

model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=1)
model.fit(X, y)
predictions = model.predict(X_test)

Your submission was successfully saved!


위 코드 셀은 데이터의 4개 속성(**"Pclass", "Sex", "SibSp", "Parch")에서 패턴을 찾습니다.  
 test.csv** 에서 승객에 대한 예측을 생성하기 전에 **train.csv* 파일의 패턴을 기반으로 random forest model 모델에서 트리를 구성합니다. 
# 3. 결과 저장
이러한 새로운 예측을 CSV 파일 **my_submission.csv**에 저장합니다.

In [ ]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions})
output.to_csv('my_submission.csv', index=False)
print("Your submission was successfully saved!")